In [18]:
# Run setup notebook from here
!git clone https://github.com/goodarzi64/GHI_Forecasting
!git pull
%run /content/GHI_Forecasting/notebooks/00_colab_setup.ipynb
%run /content/GHI_Forecasting/notebooks/01_import_datasets.ipynb

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 418 bytes | 83.00 KiB/s, done.
From https://github.com/goodarzi64/GHI_Forecasting
   877064c..f503f1d  main       -> origin/main
Updating 877064c..f503f1d
Fast-forward
 src/temporal_autoencoder.py | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)


In [28]:
import importlib
import src.temporal_autoencoder as ta
importlib.reload(ta)

# make sure tensors are float32 before training
temporal_node_tensor = torch.as_tensor(temporal_node_tensor, dtype=torch.float32, device=device)

mask_embed = torch.as_tensor(masks["mask_embed"], dtype=torch.bool, device=device)
mask_cloud = torch.as_tensor(masks["mask_cloud"], dtype=torch.bool, device=device)

model, logs, best_epoch = ta.pretrain_en_de_with_regularizers(
    train_tensor=temporal_node_tensor,
    val_tensor=None,
    in_dim=mask_embed.sum().item(),
    embed_dim=16,
    conv_hidden=128,
    window=6,
    use_attention=True,
    batch_size=8,
    lr=1e-3,
    epochs=20,
    device=device,
    early_stopping_patience=5,
    mask_embed=mask_embed,
    mask_cloud=mask_cloud,
    save_path=None,
    verbose=True,
)


AttributeError: 'int' object has no attribute 'normalize'

In [ ]:
from src.cv_splits import make_expanding_folds, extract_fold_data

T = temporal_node_tensor.shape[0]
first_train_end = int(T * 0.33)
val_window = int(T * 0.11)
n_folds = 3

folds = make_expanding_folds(
    T=T,
    train_start=0,
    first_train_end=first_train_end,
    val_window=val_window,
    n_folds=n_folds,
)

print("FOLD SPLITS:")
for i, f in enumerate(folds):
    print(f"Fold {i+1}: Train {f['train_slice']}  Val {f['val_slice']}")

train_data, val_data = extract_fold_data(temporal_node_tensor, folds, fold_idx=0)


In [ ]:
from src.hparam_search_encoder import HParamConfig, run_hparam_search, select_best

cfg = HParamConfig(
    embed_dims=[8, 16, 32],
    conv_hiddens=[32, 64, 128, 256],
    seeds=[123],
    folds=[0, 1, 2],
    window=12,
    use_attention=True,
    batch_size=8,
    epochs=20,
    lr=1e-3,
    early_stopping_patience=5,
)

results = run_hparam_search(
    temporal_node_tensor=temporal_node_tensor,
    folds=folds,
    mask_embed=masks["mask_embed"].bool().to(device),
    mask_cloud=masks["mask_cloud"].bool().to(device),
    base_dir="/content/gdrive/MyDrive/hparam_search_encoder",
    cfg=cfg,
    device=device,
    verbose=True,
)

best_cfg = select_best(results)
print(best_cfg)


In [ ]:
from src.hparam_results import collect_hparam_records, summarize_hparam_results

base_dir = "/content/gdrive/MyDrive/hparam_search_encoder"
df = collect_hparam_records(base_dir)
summary = summarize_hparam_results(df)

print(summary)
